<a href="https://colab.research.google.com/github/nasselm4i/Deep-Theoretical/blob/main/SSL_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/htdt/self-supervised.git && pip install lightly && pip install wandb
%cd self-supervised
!pip install wandb --upgrade

In [3]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Requirements

BarlowTwin - VICReg - BYOL - SimCLR - SimSiam - Data2Vec - Dino

In [5]:
import pytorch_lightning as pl
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss.vicreg_loss import VICRegLoss

from lightly.models.modules import BarlowTwinsProjectionHead
from lightly.transforms.vicreg_transform import VICRegTransform

from lightly.loss import BarlowTwinsLoss
from lightly.transforms.simclr_transform import SimCLRTransform

Whitening SSL

In [6]:
from tqdm import trange, tqdm
import numpy as np
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR, CosineAnnealingWarmRestarts
import torch.backends.cudnn as cudnn

from cfg import get_cfg
from datasets import get_ds
from methods import get_method

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# BarlowTwin

In [34]:
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import BarlowTwinsLoss
from lightly.models.modules import BarlowTwinsProjectionHead
from lightly.transforms.simclr_transform import SimCLRTransform
from sklearn.neighbors import KernelDensity

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time


wandb.init(
    project="BarlowTwin-v0-test",
    config={
        "max_epochs": 10,
        "batch_size": 256,
        "lr": 0.06
    })

config = wandb.config


# def compute_singular_values_2(model, dataloader):
# """from the dataloader and model directly """
#     device = next(model.parameters()).device
#     singular_values = []
#     with torch.no_grad():
#         for inputs, *_ in dataloader:
#             x0, x1 = inputs
#             x0 = x0.to(device)
#             x1 = x1.to(device)
#             # Get embeddings for the inputs
#             z0 = model(x0)
#             z1 = model(x1)
#             embeddings = torch.cat([z0, z1], dim=0)
#             # Compute cross-correlation matrix
#             z = embeddings.detach()
#             c = torch.matmul(z0.T, z1) / z0.shape[0]
#             # Compute singular values
#             svd = torch.svd(c)
#             singular_values.append(svd.S.cpu().numpy())
#     singular_values = np.concatenate(singular_values)
#     return singular_values

# def mutual_information(x0, z1, bandwidth=0.2, n_samples=10000):
#     """
#     Calculate the mutual information between x0 and z1 using kernel density estimation.

#     Args:
#     - x0 (np.array): Input data of shape (n_samples, n_features).
#     - z1 (np.array): Output data of shape (n_samples, n_features).
#     - bandwidth (float): Bandwidth parameter for kernel density estimation.
#     - n_samples (int): Number of samples to use for density estimation.

#     Returns:
#     - mut_info (float): Mutual information between x0 and z1.
#     """
#     with torch.no_grad():
#       x0 = x0.cpu().numpy()
#       z1 = z1.cpu().numpy()
#       n_features = x0.shape[1]
#       kde_x0 = KernelDensity(bandwidth=bandwidth).fit(x0)
#       kde_z1 = KernelDensity(bandwidth=bandwidth).fit(z1)
#       samples_x0 = kde_x0.sample(n_samples)
#       samples_z1 = kde_z1.sample(n_samples)
#       log_pdf_x0 = kde_x0.score_samples(samples_x0)
#       log_pdf_z1 = kde_z1.score_samples(samples_z1)
#       log_pdf_joint = kde_x0.score_samples(samples_z1)
#       mi = np.mean(log_pdf_x0 + log_pdf_z1 - log_pdf_joint)
#       mut_info = mi / np.log(2) / n_features
#     return mut_info

# def entropy(zX):
#     """
#     Calculate the Shannon entropy of the given embedding.

#     Args:
#     - zX (np.array): Input embedding of shape (n_samples, n_features).

#     Returns:
#     - ent (float): Shannon entropy of the embedding.
#     """
#     with torch.no_grad():
#       eps = np.finfo(float).eps
#       Zx = zX.cpu().numpy()
#       n_samples = zX.shape[0]
#       _, counts = np.unique(np.argmax(zX, axis=1), return_counts=True)
#       probs = counts / n_samples
#       ent = - np.sum(probs * np.log2(probs + eps))
#       return ent

# def entropy(zX):
#     eps = 1e-12
#     with torch.no_grad():
#         zX = torch.softmax(zX, dim=1) #each row sums up to 1
#         zX = zX.cpu().numpy() 
#         n_samples = zX.shape[0]
#         _, counts = np.unique(np.argmax(zX, axis=1), return_counts=True)
#         probs = counts / n_samples
#         ent = - np.sum(probs * np.log2(probs + eps))
#     return ent

def cross_correlation_matrix(z0, z1):
  """
    Compute the cross-correlation matrix between the embeddings z0 and z1.
    
    Args:
    - z0 (torch.Tensor): The embedding tensor for the first sequence, with shape (seq_len, embedding_dim).
    - z1 (torch.Tensor): The embedding tensor for the second sequence, with shape (seq_len, embedding_dim).
    
    Returns:
    - c (np.ndarray): the cross-correlation matrix.
    """
  with torch.no_grad():
      z0_centered = z0 - z0.mean(dim=0)
      z1_centered = z1 - z1.mean(dim=0)
      c = torch.matmul(z0_centered.T, z1_centered) / (z0_centered.shape[0] - 1)
      std0 = z0_centered.std(dim=0, unbiased=False)
      std1 = z1_centered.std(dim=0, unbiased=False)
      c = c / torch.outer(std0, std1)
      return c

def entropy(zX):
    eps = 1e-12
    with torch.no_grad():
        zX = torch.softmax(zX, dim=1)
        ent = torch.sum(-zX * torch.log2(zX + eps), dim=1)
        ent = torch.mean(ent)
    return ent.item()


def compute_singular_values(z0, z1):
    """
    Compute the singular values of the cross-correlation matrix between the embeddings z0 and z1.
    
    Args:
    - z0 (torch.Tensor): The embedding tensor for the first sequence, with shape (seq_len, embedding_dim).
    - z1 (torch.Tensor): The embedding tensor for the second sequence, with shape (seq_len, embedding_dim).
    
    Returns:
    - singular_values (np.ndarray): The singular values of the cross-correlation matrix, sorted in descending order.
    """
    with torch.no_grad():
      # Compute cross-correlation matrix
      c = cross_correlation_matrix(z0, z1)

      # Compute singular values
      svd = torch.svd(c)
      singular_values = svd.S.cpu().detach().numpy()
      return singular_values


def compute_average_off_correlation_matrix(z0,z1):
  """
    Compute the average of the off diagonal of the cross-correlation matrix between the embeddings z0 and z1.
    
    Args:
    - z0 (torch.Tensor): The embedding tensor for the first sequence, with shape (seq_len, embedding_dim).
    - z1 (torch.Tensor): The embedding tensor for the second sequence, with shape (seq_len, embedding_dim).
    
    Returns:
    - corr (float): The average of the off diagonal of the cross-correlation matrix.
    """
  with torch.no_grad():
    c = cross_correlation_matrix(z0, z1)
    corr = (c.flatten()[c.shape[0]::c.shape[0]+1].mean().item())
    return corr

class BarlowTwins(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BarlowTwins(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

cifar10 = torchvision.datasets.CIFAR10("datasets/cifar10", download=True)
transform = SimCLRTransform(input_size=32)
dataset = LightlyDataset.from_torch_dataset(cifar10, transform=transform)

collate_fn = MultiViewCollate()

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=config.batch_size,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

criterion = BarlowTwinsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config.lr)

print("Starting Training")
start = time.time()
for epoch in tqdm(range(config.max_epochs)):
    total_loss = 0
    corr = 0
    corr_count = 0
    for (x0, x1), _, _ in dataloader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(dataloader)
    avg_corr = compute_average_off_correlation_matrix(z0,z1)
    entropy_z0 = entropy(z0)
    entropy_z1 = entropy(z1)
    # mut_info = mutual_information(x0, z1, bandwidth=0.2, n_samples=10000)
    # print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}, avg cross-correlation: {avg_corr}, mutual information: {mut_info} ")
    wandb.log({"avg_loss": avg_loss, "avg_corr": avg_corr, "entropy_z0": entropy_z0, "entropy_z1": entropy_z1})

# time to train 
end = time.time()
train_time = time.strftime("%H:%M:%S", time.gmtime(end - start))
print("Time for the training :", train_time)
# compute singular values
singular_values = compute_singular_values(z0, z1)

# plot singular values
fig, ax = plt.subplots()
ax.plot(range(singular_values.size),singular_values, label=f'Singular Values')
ax.set_yscale('log')
ax.set_xlabel('Singular Value Index')
ax.set_ylabel('Singular Value')
wandb.log({"Log Singular Values ": fig})


torch.save(model.state_dict(), 'BarlowTwinV0.pth')



avg_corr,▃▅▁▃▁▆▆▄█▂
avg_loss,█▇▅▄▄▃▃▃▂▁
entropy_z0,▆▁▄▇██▇▆▇█
entropy_z1,▆▁▄▇█▇█▆▇█
epoch,▁▂▃▃▄▅▆▆▇█
avg_corr,-0.00081
avg_loss,1309.15906
entropy_z0,1.20232
entropy_z1,1.2014
epoch,9


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning:

This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Starting Training


100%|██████████| 10/10 [26:48<00:00, 160.83s/it]


00:26:48


/usr/local/lib/python3.10/dist-packages/plotly/matplotlylib/renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



In [ ]:
# class BarlowTwins(nn.Module):
#     def __init__(self, backbone):
#         super().__init__()
#         self.backbone = backbone
#         self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)

#     def forward(self, x):
#         x = self.backbone(x).flatten(start_dim=1)
#         z = self.projection_head(x)
#         return z


# resnet = torchvision.models.resnet18()
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# model = BarlowTwins(backbone)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# cifar10 = torchvision.datasets.CIFAR10("datasets/cifar10", download=True)
# transform = SimCLRTransform(input_size=32)
# dataset = LightlyDataset.from_torch_dataset(cifar10, transform=transform)

# collate_fn = MultiViewCollate()

# dataloader = torch.utils.data.DataLoader(
#     dataset,
#     batch_size=256,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
#     num_workers=8,
# )
# # classes = dataset.classes
# dataset.dataset
# for batch_idx, (views, targets, filenames) in enumerate(dataloader):
#   # Forward pass
#   print("input", views)
#   print("target", targets)

# # Define your model and optimizer
# model = MyModel()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Train loop
# for epoch in range(10):
#     for batch_idx, batch in enumerate(dataloader):
#         # Forward pass
#         inputs, targets = batch['input'], batch['target']
#         outputs = model(inputs)
#         loss = compute_loss(outputs, targets)
        
#         # Backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         # Print progress
#         if batch_idx % 1 == 0:
#             print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(inputs), len(dataloader.dataset),
#                 100. * batch_idx / len(dataloader), loss.item()))



In [ ]:
# import torch
# import torchvision
# from torch.utils.data import DataLoader
# from typing import List, Tuple

# # define the transforms to use
# transform = SimCLRTransform(input_size=32)

# # load the CIFAR10 test dataset
# cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

# # create a LightlyDataset for the CIFAR10 test dataset with the defined transform
# dataset = LightlyDataset.from_torch_dataset(cifar10_test, transform=transform)

# # create a DataLoader for the test dataset
# collate_fn = MultiViewCollate()
# batch_size = 256  # adjust this according to your available memory
# num_workers = 8  # adjust this according to your available CPU resources
# test_loader = torch.utils.data.DataLoader(
#     dataset,
#     batch_size=batch_size,
#     collate_fn=collate_fn,
#     shuffle=False,
#     num_workers=num_workers,
#     drop_last=False
# )

# # load your trained Barlow Twin model
# checkpoint = torch.load('BarlowTwinV0.pth')

# resnet = torchvision.models.resnet18()
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# model = BarlowTwins(backbone)
# model.load_state_dict(checkpoint)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # set the model to evaluation mode
# model.eval()

# # define a function to compute the accuracy of the model on the test dataset
# # def test(model: torch.nn.Module, test_loader: torch.utils.data.DataLoader) -> float:
# #     num_correct = 0
# #     num_total = 0
# #     with torch.no_grad():
# #         for views, labels, filenames in test_loader:
# #             # move the views and labels to the device used for training the model
# #             views = [view.to(device) for view in views]
# #             labels = labels.to(device)

# #             # compute the forward pass of the model
# #             projections = model(views)

# #             # compute the cosine similarity between the projections of the views
# #             sim_matrix = torch.matmul(projections, projections.t())
# #             sim_matrix = sim_matrix - torch.diag(sim_matrix.diag())
# #             sim_matrix = sim_matrix / torch.norm(sim_matrix, dim=1, keepdim=True)

# #             # predict the label by selecting the view with the highest average cosine similarity
# #             pred = sim_matrix.mean(1).argmax().item()

# #             # compute the number of correctly classified images
# #             num_correct += int(pred == labels.item())

# #             # update the total number of images
# #             num_total += 1

# #     # return the accuracy of the model on the test dataset
# #     return num_correct / num_total

# def test(model: torch.nn.Module, test_loader: torch.utils.data.DataLoader) -> float:
#     num_correct = 0
#     num_total = 0
#     with torch.no_grad():
#         for views, labels, filenames in test_loader:
#             # move the views and labels to the device used for training the model
#             views = [view.to(device) for view in views]
#             labels = labels.to(device)

#             # compute the forward pass of the model
#             projections = model(views)

#             # compute the cosine similarity between the projections of the views
#             sim_matrix = torch.matmul(projections, projections.t())
#             sim_matrix = sim_matrix - torch.diag(sim_matrix.diag())
#             sim_matrix = sim_matrix / torch.norm(sim_matrix, dim=1, keepdim=True)

#             # predict the label by selecting the view with the highest average cosine similarity
#             pred = sim_matrix.mean(1).argmax(dim=0)  # compute predicted labels for the batch

#             # compute the number of correctly classified images
#             num_correct += (pred == labels).sum().item()

#             # update the total number of images
#             num_total += len(labels)

#     # return the accuracy of the model on the test dataset
#     return num_correct / num_total

# # calculate the accuracy of the model on the test dataset
# accuracy = test(model, test_loader)
# print(f'Accuracy: {accuracy:.2f}')

In [41]:
model = torch.load('BarlowTwinV0.pth')
print(model.keys())

odict_keys(['backbone.0.weight', 'backbone.1.weight', 'backbone.1.bias', 'backbone.1.running_mean', 'backbone.1.running_var', 'backbone.1.num_batches_tracked', 'backbone.4.0.conv1.weight', 'backbone.4.0.bn1.weight', 'backbone.4.0.bn1.bias', 'backbone.4.0.bn1.running_mean', 'backbone.4.0.bn1.running_var', 'backbone.4.0.bn1.num_batches_tracked', 'backbone.4.0.conv2.weight', 'backbone.4.0.bn2.weight', 'backbone.4.0.bn2.bias', 'backbone.4.0.bn2.running_mean', 'backbone.4.0.bn2.running_var', 'backbone.4.0.bn2.num_batches_tracked', 'backbone.4.1.conv1.weight', 'backbone.4.1.bn1.weight', 'backbone.4.1.bn1.bias', 'backbone.4.1.bn1.running_mean', 'backbone.4.1.bn1.running_var', 'backbone.4.1.bn1.num_batches_tracked', 'backbone.4.1.conv2.weight', 'backbone.4.1.bn2.weight', 'backbone.4.1.bn2.bias', 'backbone.4.1.bn2.running_mean', 'backbone.4.1.bn2.running_var', 'backbone.4.1.bn2.num_batches_tracked', 'backbone.5.0.conv1.weight', 'backbone.5.0.bn1.weight', 'backbone.5.0.bn1.bias', 'backbone.5.0.b

In [ ]:
# create a linear classifier
class LinearClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.fc(x)
        return out

# initialize the linear classifier
linear_classifier = LinearClassifier(input_size=barlow_twins_model.embedding_size, output_size=num_classes)

# define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(linear_classifier.parameters(), lr=learning_rate)

# train the linear classifier
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # forward pass
        embeddings = barlow_twins_model(images)
        outputs = linear_classifier(embeddings)
        loss = criterion(outputs, labels)
        
        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# evaluate the linear classifier
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # compute the embeddings using the Barlow Twins model
        embeddings = barlow_twins_model(images)
        
        # compute the outputs using the linear classifier
        outputs = linear_classifier(embeddings)
        _, predicted = torch.max(outputs.data, 1)
        
        # update the accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = 100 * correct / total
print('Accuracy: {:.2f}%'.format(accuracy))


# VICReg

- Changing the 

In [ ]:
wandb.init(
    project="BarlowTwin-v0-test",
    config={
        "max_epochs": 10,
        "batch_size": 256,
        "lr": 0.06
    })

config = wandb.config

class VICReg(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)
        self.criterion = VICRegLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=0.06)
        return optim


model = VICReg()

cifar10 = torchvision.datasets.CIFAR10("datasets/cifar10", download=True)
dataset = LightlyDataset.from_torch_dataset(cifar10, VICRegTransform(input_size=32))
# or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder")

collate_fn = MultiViewCollate()

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

accelerator = "gpu" if torch.cuda.is_available() else "cpu"

trainer = pl.Trainer(max_epochs=10, devices=1, accelerator=accelerator)
trainer.fit(model=model, train_dataloaders=dataloader)

100%|██████████| 170498071/170498071 [00:06<00:00, 25816745.10it/s]


Extracting datasets/cifar10/cifar-10-python.tar.gz to datasets/cifar10


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type                      | Params
--------------------------------------------------------------
0 | backbone        | Sequential       

Training: 0it [00:00, ?it/s]

# Whitening Self Supervised Learning (Building in Progress)

\begin{gathered}
\min _\theta \mathbb{E}\left[\operatorname{dist}\left(\mathbf{z}_i, \mathbf{z}_j\right)\right], \\
\text { s.t. } \operatorname{cov}\left(\mathbf{z}_i, \mathbf{z}_i\right)=\operatorname{cov}\left(\mathbf{z}_j, \mathbf{z}_j\right)=I,
\end{gathered}

$$
\begin{gathered}
\operatorname{dist}\left(\mathbf{z}_i, \mathbf{z}_j\right)=\left\|\frac{\mathbf{z}_i}{\left\|\mathbf{z}_i\right\|_2}-\frac{\mathbf{z}_j}{\left\|\mathbf{z}_j\right\|_2}\right\|_2^2 \\
=2-2 \frac{\left\langle\mathbf{z}_i, \mathbf{z}_j\right\rangle}{\left\|\mathbf{z}_i\right\|_2 \cdot\left\|\mathbf{z}_j\right\|_2}
\end{gathered}
$$
$$
L_{W-M S E}(V)=\frac{2}{N d(d-1)} \sum \operatorname{dist}\left(\mathbf{z}_i, \mathbf{z}_j\right)
$$
where the sum is over $\left(\mathbf{v}_i, \mathbf{v}_j\right) \in V, \operatorname{pos}(i, j)=$ true, $\mathbf{z}=$ $\operatorname{Whitening}(\mathbf{v})$, and:
$\operatorname{Whitening}(\mathbf{v})=W_V\left(\mathbf{v}-\boldsymbol{\mu}_V\right)$.
$\boldsymbol{\mu}_V$ is the mean of the elements in $V: \boldsymbol{\mu}_V=$ $\frac{1}{K} \sum_k \mathbf{v}_k$, while the matrix $W_V$ is such that: $W_V^{\top} W_V=$ $\Sigma_V^{-1}$, being $\Sigma_V$ the covariance matrix of $V$ :
$$
\Sigma_V=\frac{1}{K-1} \sum_k\left(\mathbf{v}_k-\boldsymbol{\mu}_V\right)\left(\mathbf{v}_k-\boldsymbol{\mu}_V\right)^T
$$
We provide here the equations for whitening differentiation as reported in (Siarohin et al., 2019). Let $Z$ be the whitened version of the batch $V$, i.e., $Z=W_V\left(V-\boldsymbol{\mu}_V\right)$. The gradient $\frac{\partial L}{\partial V}$ can be computed by:
$$
\frac{\partial L}{\partial V}=\frac{2}{K-1} \frac{\partial L}{\partial \Sigma} V+W_V^T \frac{\partial L}{\partial Z}
$$
where the partial derivative $\frac{\partial L}{\partial Z}$ is backpropogated, while $\frac{\partial L}{\partial \Sigma}$ is computed as follows:
$$
\frac{\partial L}{\partial \Sigma}=-\frac{1}{2} W_V^T\left(P \circ \frac{\partial L}{\partial W_V} W_V^T+\left(P \circ \frac{\partial L}{\partial W_V} W_V^T\right)^T\right) W_V
$$
In (10), $\circ$ is Hadamard product, while $\frac{\partial L}{\partial W_V}$ is:
$$
\frac{\partial L}{\partial W_V}=\frac{\partial L}{\partial Z} V^T
$$
and $P$ is:
$$
P=\left(\begin{array}{cccc}
\frac{1}{2} & 0 & \cdots & 0 \\
1 & \frac{1}{2} & \ddots & 0 \\
1 & \ddots & \ddots & 0 \\
1 & \cdots & 1 & \frac{1}{2}
\end{array}\right)
$$

#### W-MSE 4
```
python -m train --dataset cifar10 --epoch 1000 --lr 3e-3 --num_samples 4 --bs 256 --emb 64 --w_size 128
python -m train --dataset cifar100 --epoch 1000 --lr 3e-3 --num_samples 4 --bs 256 --emb 64 --w_size 128
python -m train --dataset stl10 --epoch 2000 --lr 2e-3 --num_samples 4 --bs 256 --emb 128 --w_size 256
python -m train --dataset tiny_in --epoch 1000 --lr 2e-3 --num_samples 4 --bs 256 --emb 128 --w_size 256
```

#### W-MSE 2
```
python -m train --dataset cifar10 --epoch 1000 --lr 3e-3 --emb 64 --w_size 128
python -m train --dataset cifar100 --epoch 1000 --lr 3e-3 --emb 64 --w_size 128
python -m train --dataset stl10 --epoch 2000 --lr 2e-3 --emb 128 --w_size 256 --w_iter 4
python -m train --dataset tiny_in --epoch 1000 --lr 2e-3 --emb 128 --w_size 256 --w_iter 4
```

#### Contrastive
```
python -m train --dataset cifar10 --epoch 1000 --lr 3e-3 --emb 64 --method contrastive
python -m train --dataset cifar100 --epoch 1000 --lr 3e-3 --emb 64 --method contrastive
python -m train --dataset stl10 --epoch 2000 --lr 2e-3 --emb 128 --method contrastive
python -m train --dataset tiny_in --epoch 1000 --lr 2e-3 --emb 128 --method contrastive
```

#### BYOL
```
python -m train --dataset cifar10 --epoch 1000 --lr 3e-3 --emb 64 --method byol
python -m train --dataset cifar100 --epoch 1000 --lr 3e-3 --emb 64 --method byol
python -m train --dataset stl10 --epoch 2000 --lr 2e-3 --emb 128 --method byol
python -m train --dataset tiny_in --epoch 1000 --lr 2e-3 --emb 128 --method byol
```

#### ImageNet-100
```
python -m train --dataset imagenet --epoch 240 --lr 2e-3 --emb 128 --w_size 256 --crop_s0 0.08 --cj0 0.8 --cj1 0.8 --cj2 0.8 --cj3 0.2 --gs_p 0.2
python -m train --dataset imagenet --epoch 240 --lr 2e-3 --num_samples 4 --bs 256 --emb 128 --w_size 256 --crop_s0 0.08 --cj0 0.8 --cj1 0.8 --cj2 0.8 --cj3 0.2 --gs_p 0.2
```

Use `--no_norm` to disable normalization (for Euclidean distance).


Press (2) Existing account \\
API KEY = 99c1b4fb29042df88f0099033d45baa2f380d34d

In [ ]:
!python -m train --dataset cifar10 --epoch 1000 --lr 3e-3 --num_samples 4 --bs 256 --emb 64 --w_size 128


  2% 3/195 [00:05<06:04,  1.90s/it]
  2% 4/195 [00:06<04:41,  1.47s/it]
  3% 5/195 [00:08<05:00,  1.58s/it]
  3% 6/195 [00:09<04:10,  1.32s/it]
  4% 7/195 [00:10<04:04,  1.30s/it]
  4% 8/195 [00:11<03:34,  1.15s/it]
  5% 9/195 [00:12<03:21,  1.08s/it]
  5% 10/195 [00:13<03:05,  1.00s/it]
  6% 11/195 [00:14<03:22,  1.10s/it]
  6% 12/195 [00:15<03:05,  1.01s/it]
  7% 13/195 [00:16<03:13,  1.06s/it]
  7% 14/195 [00:17<02:57,  1.02it/s]
  8% 15/195 [00:18<03:20,  1.12s/it]
  8% 16/195 [00:19<03:05,  1.03s/it]
  9% 17/195 [00:22<04:32,  1.53s/it]
  9% 18/195 [00:23<03:52,  1.32s/it]
 10% 19/195 [00:24<03:59,  1.36s/it]
 10% 20/195 [00:25<03:28,  1.19s/it]
 11% 21/195 [00:26<03:27,  1.19s/it]
 11% 22/195 [00:27<03:05,  1.07s/it]
 12% 23/195 [00:28<03:20,  1.16s/it]
 12% 24/195 [00:29<03:01,  1.06s/it]
 13% 25/195 [00:30<03:12,  1.13s/it]
 13% 26/195 [00:31<02:55,  1.04s/it]
 14% 27/195 [00:32<02:59,  1.07s/it]
 14% 28/195 [00:33<02:48,  1.01s/it]
 15% 29/195 [00:35<03:42,  1.34s/it]
 15% 30

# BYOL

In [ ]:
wandb.init(
    project="BarlowTwin-v0-test",
    config={
        "max_epochs": 10,
        "batch_size": 256,
        "lr": 0.06
    })

config = wandb.config


import copy

import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLPredictionHead, BYOLProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.utils.scheduler import cosine_schedule


class BYOL(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = BYOLProjectionHead(512, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BYOL(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

cifar10 = torchvision.datasets.CIFAR10("datasets/cifar10", download=True)
transform = SimCLRTransform(input_size=32)
dataset = LightlyDataset.from_torch_dataset(cifar10, transform=transform)
# or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder")

collate_fn = MultiViewCollate()

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

epochs = 10

print("Starting Training")
for epoch in range(epochs):
    total_loss = 0
    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    for (x0, x1), _, _ in dataloader:
        update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
        update_momentum(
            model.projection_head, model.projection_head_momentum, m=momentum_val
        )
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

# SimSiam

In [ ]:
import torch
import torchvision
from torch import nn

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import SimSiamTransform


class SimSiam(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SimSiamProjectionHead(512, 512, 128)
        self.prediction_head = SimSiamPredictionHead(128, 64, 128)

    def forward(self, x):
        f = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(f)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = SimSiam(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

cifar10 = torchvision.datasets.CIFAR10("datasets/cifar10", download=True)
transform = SimSiamTransform(input_size=32)
dataset = LightlyDataset.from_torch_dataset(cifar10, transform=transform)

collate_fn = MultiViewCollate()

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

print("Starting Training")
for epoch in range(10):
    total_loss = 0
    for (x0, x1), _, _ in dataloader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0, p0 = model(x0)
        z1, p1 = model(x1)
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

# Test Accuracy of the models 



In [51]:
!pip install pytorch-lightning --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
# -*- coding: utf-8 -*-
"""
Benchmark Results

Updated: 27.03.2023 (42a6a924b1b6d5b6cc89a6b2a0a0942cc4af93ab)

------------------------------------------------------------------------------------------
| Model         | Batch Size | Epochs |  KNN Test Accuracy |       Time | Peak GPU Usage |
------------------------------------------------------------------------------------------
| BarlowTwins   |        128 |    200 |              0.842 |  375.9 Min |      1.7 GByte |
| BYOL          |        128 |    200 |              0.869 |  121.9 Min |      1.6 GByte |
| DCL           |        128 |    200 |              0.844 |  102.2 Min |      1.5 GByte |
| DCLW          |        128 |    200 |              0.833 |  100.4 Min |      1.5 GByte |
| DINO          |        128 |    200 |              0.840 |  120.3 Min |      1.6 GByte |
| FastSiam      |        128 |    200 |              0.906 |  164.0 Min |      2.7 GByte |
| Moco          |        128 |    200 |              0.838 |  128.8 Min |      1.7 GByte |
| NNCLR         |        128 |    200 |              0.834 |  101.5 Min |      1.5 GByte |
| SimCLR        |        128 |    200 |              0.847 |   97.7 Min |      1.5 GByte |
| SimSiam       |        128 |    200 |              0.819 |   97.3 Min |      1.6 GByte |
| SwaV          |        128 |    200 |              0.812 |   99.6 Min |      1.5 GByte |
| SMoG          |        128 |    200 |              0.743 |  192.2 Min |      1.2 GByte |
------------------------------------------------------------------------------------------
| BarlowTwins   |        512 |    200 |              0.819 |  153.3 Min |      5.1 GByte |
| BYOL          |        512 |    200 |              0.868 |  108.3 Min |      5.6 GByte |
| DCL           |        512 |    200 |              0.840 |   88.2 Min |      4.9 GByte |
| DCLW          |        512 |    200 |              0.824 |   87.9 Min |      4.9 GByte |
| DINO          |        512 |    200 |              0.813 |  108.6 Min |      5.0 GByte |
| FastSiam      |        512 |    200 |              0.788 |  146.9 Min |      9.5 GByte |
| Moco (*)      |        512 |    200 |              0.847 |  112.2 Min |      5.6 GByte |
| NNCLR (*)     |        512 |    200 |              0.815 |   88.1 Min |      5.0 GByte |
| SimCLR        |        512 |    200 |              0.848 |   87.1 Min |      4.9 GByte |
| SimSiam       |        512 |    200 |              0.764 |   87.8 Min |      5.0 GByte |
| SwaV          |        512 |    200 |              0.842 |   88.7 Min |      4.9 GByte |
| SMoG          |        512 |    200 |              0.686 |  110.0 Min |      3.4 GByte |
------------------------------------------------------------------------------------------
| BarlowTwins   |        512 |    800 |              0.859 |  517.5 Min |      7.9 GByte |
| BYOL          |        512 |    800 |              0.910 |  400.9 Min |      5.4 GByte |
| DCL           |        512 |    800 |              0.874 |  334.6 Min |      4.9 GByte |
| DCLW          |        512 |    800 |              0.871 |  333.3 Min |      4.9 GByte |
| DINO          |        512 |    800 |              0.848 |  405.2 Min |      5.0 GByte |
| FastSiam      |        512 |    800 |              0.902 |  582.0 Min |      9.5 GByte |
| Moco (*)      |        512 |    800 |              0.899 |  417.8 Min |      5.4 GByte |
| NNCLR (*)     |        512 |    800 |              0.892 |  335.0 Min |      5.0 GByte |
| SimCLR        |        512 |    800 |              0.879 |  331.1 Min |      4.9 GByte |
| SimSiam       |        512 |    800 |              0.904 |  333.7 Min |      5.1 GByte |
| SwaV          |        512 |    800 |              0.884 |  330.5 Min |      5.0 GByte |
| SMoG          |        512 |    800 |              0.800 |  415.6 Min |      3.2 GByte |
------------------------------------------------------------------------------------------

(*): Increased size of memory bank from 4096 to 8192 to avoid too quickly 
changing memory bank due to larger batch size.

The benchmarks were created on a single NVIDIA RTX A6000.

Note that this benchmark also supports a multi-GPU setup. If you run it on
a system with multiple GPUs make sure that you kill all the processes when
killing the application. Due to the way we setup this benchmark the distributed
processes might continue the benchmark if one of the nodes is killed.
If you know how to fix this don't hesitate to create an issue or PR :)

"""
import copy
import os
import time

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision
from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.optimizers.lars import LARS

from lightly.data import LightlyDataset
from lightly.data.multi_view_collate import MultiViewCollate
from lightly.loss import (
    BarlowTwinsLoss,
    DCLLoss,
    DCLWLoss,
    DINOLoss,
    NegativeCosineSimilarity,
    NTXentLoss,
    SwaVLoss,
    VICRegLLoss,
    VICRegLoss,
    memory_bank,
)
from lightly.models import ResNetGenerator, modules, utils
from lightly.models.modules import heads
from lightly.transforms import (
    DINOTransform,
    FastSiamTransform,
    SimCLRTransform,
    SimSiamTransform,
    SMoGTransform,
    SwaVTransform,
    VICRegLTransform,
    VICRegTransform,
)
from lightly.transforms.utils import IMAGENET_NORMALIZE
from lightly.utils import scheduler
from lightly.utils.benchmarking import BenchmarkModule

logs_root_dir = os.path.join(os.getcwd(), "benchmark_logs")

# set max_epochs to 800 for long run (takes around 10h on a single V100)
max_epochs = 200
num_workers = 8
knn_k = 200
knn_t = 0.1
classes = 10

# Set to True to enable Distributed Data Parallel training.
distributed = False

# Set to True to enable Synchronized Batch Norm (requires distributed=True).
# If enabled the batch norm is calculated over all gpus, otherwise the batch
# norm is only calculated from samples on the same gpu.
sync_batchnorm = False

# Set to True to gather features from all gpus before calculating
# the loss (requires distributed=True).
# If enabled then the loss on every gpu is calculated with features from all
# gpus, otherwise only features from the same gpu are used.
gather_distributed = False

# benchmark
n_runs = 1  # optional, increase to create multiple runs and report mean + std
batch_size = 128
lr_factor = batch_size / 128  # scales the learning rate linearly with batch size

# Number of devices and hardware to use for training.
devices = torch.cuda.device_count() if torch.cuda.is_available() else 1
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

if distributed:
    strategy = "ddp"
    # reduce batch size for distributed training
    batch_size = batch_size // devices
else:
    strategy = None  # Set to "auto" if using PyTorch Lightning >= 2.0
    # limit to single device if not using distributed training
    devices = min(devices, 1)

# Adapted from our MoCo Tutorial on CIFAR-10
#
# Replace the path with the location of your CIFAR-10 dataset.
# We assume we have a train folder with subfolders
# for each class and .png images inside.
#
# You can download `CIFAR-10 in folders from kaggle
# <https://www.kaggle.com/swaroopkml/cifar10-pngs-in-folders>`_.

# The dataset structure should be like this:
# cifar10/train/
#  L airplane/
#    L 10008_airplane.png
#    L ...
#  L automobile/
#  L bird/
#  L cat/
#  L deer/
#  L dog/
#  L frog/
#  L horse/
#  L ship/
#  L truck/
path_to_train = "/datasets/cifar10/train/"
path_to_test = "/datasets/cifar10/test/"

# Collate function init
collate_fn = MultiViewCollate()

# Use SimCLR augmentations
simclr_transform = SimCLRTransform(
    input_size=32,
    cj_strength=0.5,
    gaussian_blur=0.0,
)

# Use SimSiam augmentations
simsiam_transform = SimSiamTransform(
    input_size=32,
    gaussian_blur=0.0,
)

# Multi crop augmentation for FastSiam
fast_siam_transform = FastSiamTransform(input_size=32, gaussian_blur=0.0)

# Multi crop augmentation for SwAV, additionally, disable blur for cifar10
swav_transform = SwaVTransform(
    crop_sizes=[32],
    crop_counts=[2],  # 2 crops @ 32x32px
    crop_min_scales=[0.14],
    cj_strength=0.5,
    gaussian_blur=0,
)

# Multi crop augmentation for DINO, additionally, disable blur for cifar10
dino_transform = DINOTransform(
    global_crop_size=32,
    n_local_views=0,
    cj_strength=0.5,
    gaussian_blur=(0, 0, 0),
)

# Two crops for SMoG
smog_transform = SMoGTransform(
    crop_sizes=(32, 32),
    crop_counts=(1, 1),
    cj_strength=0.5,
    gaussian_blur_probs=(0.0, 0.0),
    crop_min_scales=(0.2, 0.2),
    crop_max_scales=(1.0, 1.0),
)

vicreg_transform = VICRegTransform(
    input_size=32,
    cj_strength=0.5,
)
# Transform  passing geometrical transformation for VICRegL
vicregl_transform = VICRegLTransform(
    global_crop_size=128,
    n_local_views=0,
    global_grid_size=4,
    cj_strength=0.5,
)

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=IMAGENET_NORMALIZE["mean"],
            std=IMAGENET_NORMALIZE["std"],
        ),
    ]
)

# we use test transformations for getting the feature for kNN on train data
dataset_train_kNN = LightlyDataset(input_dir=path_to_train, transform=test_transforms)

dataset_test = LightlyDataset(input_dir=path_to_test, transform=test_transforms)


def create_dataset_train_ssl(model):
    """Helper method to apply the correct transform for ssl.

    Args:
        model:
            Model class for which to select the transform.
    """
    model_to_transform = {
        BarlowTwinsModel: simclr_transform,
        BYOLModel: simclr_transform,
        DCL: simclr_transform,
        DCLW: simclr_transform,
        DINOModel: dino_transform,
        FastSiamModel: fast_siam_transform,
        MocoModel: simclr_transform,
        NNCLRModel: simclr_transform,
        SimCLRModel: simclr_transform,
        SimSiamModel: simsiam_transform,
        SwaVModel: swav_transform,
        SMoGModel: smog_transform,
        VICRegModel: vicreg_transform,
        VICRegLModel: vicregl_transform,
    }
    transform = model_to_transform[model]
    return LightlyDataset(input_dir=path_to_train, transform=transform)


def get_data_loaders(batch_size: int, dataset_train_ssl):
    """Helper method to create dataloaders for ssl, kNN train and kNN test.

    Args:
        batch_size: Desired batch size for all dataloaders.
    """
    dataloader_train_ssl = torch.utils.data.DataLoader(
        dataset_train_ssl,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=num_workers,
    )

    dataloader_train_kNN = torch.utils.data.DataLoader(
        dataset_train_kNN,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=num_workers,
    )

    dataloader_test = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=num_workers,
    )

    return dataloader_train_ssl, dataloader_train_kNN, dataloader_test


class MocoModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)

        # create a ResNet backbone and remove the classification head
        num_splits = 0 if sync_batchnorm else 8
        resnet = ResNetGenerator("resnet-18", num_splits=num_splits)
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )

        # create a moco model based on ResNet
        self.projection_head = heads.MoCoProjectionHead(512, 512, 128)
        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)
        utils.deactivate_requires_grad(self.backbone_momentum)
        utils.deactivate_requires_grad(self.projection_head_momentum)

        # create our loss with the optional memory bank
        self.criterion = NTXentLoss(
            temperature=0.1,
            memory_bank_size=4096,
        )

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        return self.projection_head(x)

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch

        # update momentum
        utils.update_momentum(self.backbone, self.backbone_momentum, 0.99)
        utils.update_momentum(self.projection_head, self.projection_head_momentum, 0.99)

        def step(x0_, x1_):
            x1_, shuffle = utils.batch_shuffle(x1_, distributed=distributed)
            x0_ = self.backbone(x0_).flatten(start_dim=1)
            x0_ = self.projection_head(x0_)

            x1_ = self.backbone_momentum(x1_).flatten(start_dim=1)
            x1_ = self.projection_head_momentum(x1_)
            x1_ = utils.batch_unshuffle(x1_, shuffle, distributed=distributed)
            return x0_, x1_

        # We use a symmetric loss (model trains faster at little compute overhead)
        # https://colab.research.google.com/github/facebookresearch/moco/blob/colab-notebook/colab/moco_cifar10_demo.ipynb
        loss_1 = self.criterion(*step(x0, x1))
        loss_2 = self.criterion(*step(x1, x0))

        loss = 0.5 * (loss_1 + loss_2)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        params = list(self.backbone.parameters()) + list(
            self.projection_head.parameters()
        )
        optim = torch.optim.SGD(
            params,
            lr=6e-2 * lr_factor,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class SimCLRModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.projection_head = heads.SimCLRProjectionHead(512, 512, 128)
        self.criterion = NTXentLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2 * lr_factor, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class SimSiamModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.prediction_head = heads.SimSiamPredictionHead(2048, 512, 2048)
        # use a 2-layer projection head for cifar10 as described in the paper
        self.projection_head = heads.ProjectionHead(
            [
                (512, 2048, nn.BatchNorm1d(2048), nn.ReLU(inplace=True)),
                (2048, 2048, nn.BatchNorm1d(2048), None),
            ]
        )
        self.criterion = NegativeCosineSimilarity()

    def forward(self, x):
        f = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(f)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0, p0 = self.forward(x0)
        z1, p1 = self.forward(x1)
        loss = 0.5 * (self.criterion(z0, p1) + self.criterion(z1, p0))
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(),
            lr=6e-2,  # no lr-scaling, results in better training stability
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class FastSiamModel(SimSiamModel):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)

    def training_step(self, batch, batch_idx):
        views, _, _ = batch
        features = [self.forward(view) for view in views]
        zs = torch.stack([z for z, _ in features])
        ps = torch.stack([p for _, p in features])

        loss = 0.0
        for i in range(len(views)):
            mask = torch.arange(len(views), device=self.device) != i
            loss += self.criterion(ps[i], torch.mean(zs[mask], dim=0)) / len(views)

        self.log("train_loss_ssl", loss)
        return loss


class BarlowTwinsModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        # use a 2-layer projection head for cifar10 as described in the paper
        self.projection_head = heads.ProjectionHead(
            [
                (512, 2048, nn.BatchNorm1d(2048), nn.ReLU(inplace=True)),
                (2048, 2048, None, None),
            ]
        )

        self.criterion = BarlowTwinsLoss(gather_distributed=gather_distributed)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2 * lr_factor, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class BYOLModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )

        # create a byol model based on ResNet
        self.projection_head = heads.BYOLProjectionHead(512, 1024, 256)
        self.prediction_head = heads.BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        utils.deactivate_requires_grad(self.backbone_momentum)
        utils.deactivate_requires_grad(self.projection_head_momentum)

        self.criterion = NegativeCosineSimilarity()

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z

    def training_step(self, batch, batch_idx):
        utils.update_momentum(self.backbone, self.backbone_momentum, m=0.99)
        utils.update_momentum(
            self.projection_head, self.projection_head_momentum, m=0.99
        )
        (x0, x1), _, _ = batch
        p0 = self.forward(x0)
        z0 = self.forward_momentum(x0)
        p1 = self.forward(x1)
        z1 = self.forward_momentum(x1)
        loss = 0.5 * (self.criterion(p0, z1) + self.criterion(p1, z0))
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        params = (
            list(self.backbone.parameters())
            + list(self.projection_head.parameters())
            + list(self.prediction_head.parameters())
        )
        optim = torch.optim.SGD(
            params,
            lr=6e-2 * lr_factor,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class VICRegModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = heads.BarlowTwinsProjectionHead(512, 2048, 2048)
        self.criterion = VICRegLoss()
        self.warmup_epochs = 40 if max_epochs >= 800 else 20

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        # Training diverges without LARS
        optim = LARS(
            self.parameters(),
            lr=0.3 * lr_factor,
            weight_decay=1e-4,
            momentum=0.9,
        )
        cosine_scheduler = scheduler.CosineWarmupScheduler(
            optim, self.warmup_epochs, max_epochs
        )
        return [optim], [cosine_scheduler]


class VICRegLModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = torchvision.models.resnet18()

        # The train_backbone variable is introduced in order to fit with the
        # structure of BenchmarkModule. During training, train_backbone is used
        # to extract local and global features. Durig evaluation, backbone is used
        # to evaluate global features.
        self.train_backbone = nn.Sequential(*list(resnet.children())[:-2])
        self.projection_head = heads.BarlowTwinsProjectionHead(512, 2048, 2048)
        self.local_projection_head = heads.VicRegLLocalProjectionHead(512, 128, 128)
        self.average_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.criterion = VICRegLLoss(num_matches=(16, 4))
        self.backbone = nn.Sequential(self.train_backbone, self.average_pool)
        self.warmup_epochs = 20 if max_epochs >= 800 else 10

    def forward(self, x):
        x = self.train_backbone(x)
        y = self.average_pool(x).flatten(start_dim=1)
        z = self.projection_head(y)
        y_local = x.permute(0, 2, 3, 1)  # (B, D, H, W) to (B, H, W, D)
        z_local = self.local_projection_head(y_local)
        return z, z_local

    def training_step(self, batch, batch_index):
        views_and_grids = batch[0]
        views = views_and_grids[: len(views_and_grids) // 2]
        grids = views_and_grids[len(views_and_grids) // 2 :]
        features = [self.forward(view) for view in views]
        loss = self.criterion(
            global_view_features=features[:2],
            global_view_grids=grids[:2],
            local_view_features=features[2:],
            local_view_grids=grids[2:],
        )
        return loss

    def configure_optimizers(self):
        # Training diverges without LARS
        optim = LARS(
            self.parameters(),
            lr=0.3 * lr_factor,
            weight_decay=1e-4,
            momentum=0.9,
        )
        cosine_scheduler = scheduler.CosineWarmupScheduler(
            optim, self.warmup_epochs, max_epochs
        )
        return [optim], [cosine_scheduler]


# class SwaVModel(BenchmarkModule):
#     def __init__(self, dataloader_kNN, num_classes):
#         super().__init__(dataloader_kNN, num_classes)
#         # create a ResNet backbone and remove the classification head
#         resnet = ResNetGenerator("resnet-18")
#         self.backbone = nn.Sequential(
#             *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
#         )

#         self.projection_head = heads.SwaVProjectionHead(512, 512, 128)
#         self.prototypes = heads.SwaVPrototypes(128, 512)  # use 512 prototypes

#         self.criterion = SwaVLoss(sinkhorn_gather_distributed=gather_distributed)

#     def forward(self, x):
#         x = self.backbone(x).flatten(start_dim=1)
#         x = self.projection_head(x)
#         x = nn.functional.normalize(x, dim=1, p=2)
#         return self.prototypes(x)

#     def training_step(self, batch, batch_idx):
#         # normalize the prototypes so they are on the unit sphere
#         self.prototypes.normalize()

#         # the multi-crop dataloader returns a list of image crops where the
#         # first two items are the high resolution crops and the rest are low
#         # resolution crops
#         multi_crops, _, _ = batch
#         multi_crop_features = [self.forward(x) for x in multi_crops]

#         # split list of crop features into high and low resolution
#         high_resolution_features = multi_crop_features[:2]
#         low_resolution_features = multi_crop_features[2:]

#         # calculate the SwaV loss
#         loss = self.criterion(high_resolution_features, low_resolution_features)

#         self.log("train_loss_ssl", loss)
#         return loss

#     def configure_optimizers(self):
#         optim = torch.optim.Adam(
#             self.parameters(),
#             lr=1e-3 * lr_factor,
#             weight_decay=1e-6,
#         )
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
#         return [optim], [scheduler]


class NNCLRModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.prediction_head = heads.NNCLRPredictionHead(256, 4096, 256)
        # use only a 2-layer projection head for cifar10
        self.projection_head = heads.ProjectionHead(
            [
                (512, 2048, nn.BatchNorm1d(2048), nn.ReLU(inplace=True)),
                (2048, 256, nn.BatchNorm1d(256), None),
            ]
        )

        self.criterion = NTXentLoss()
        self.memory_bank = modules.NNMemoryBankModule(size=4096)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0, p0 = self.forward(x0)
        z1, p1 = self.forward(x1)
        z0 = self.memory_bank(z0, update=False)
        z1 = self.memory_bank(z1, update=True)
        loss = 0.5 * (self.criterion(z0, p1) + self.criterion(z1, p0))
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(),
            lr=6e-2 * lr_factor,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class DINOModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.head = self._build_projection_head()
        self.teacher_backbone = copy.deepcopy(self.backbone)
        self.teacher_head = self._build_projection_head()

        utils.deactivate_requires_grad(self.teacher_backbone)
        utils.deactivate_requires_grad(self.teacher_head)

        self.criterion = DINOLoss(output_dim=2048)

    def _build_projection_head(self):
        head = heads.DINOProjectionHead(512, 2048, 256, 2048, batch_norm=True)
        # use only 2 layers for cifar10
        head.layers = heads.ProjectionHead(
            [
                (512, 2048, nn.BatchNorm1d(2048), nn.GELU()),
                (2048, 256, None, None),
            ]
        ).layers
        return head

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.head(y)
        return z

    def forward_teacher(self, x):
        y = self.teacher_backbone(x).flatten(start_dim=1)
        z = self.teacher_head(y)
        return z

    def training_step(self, batch, batch_idx):
        utils.update_momentum(self.backbone, self.teacher_backbone, m=0.99)
        utils.update_momentum(self.head, self.teacher_head, m=0.99)
        views, _, _ = batch
        views = [view.to(self.device) for view in views]
        global_views = views[:2]
        teacher_out = [self.forward_teacher(view) for view in global_views]
        student_out = [self.forward(view) for view in views]
        loss = self.criterion(teacher_out, student_out, epoch=self.current_epoch)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        param = list(self.backbone.parameters()) + list(self.head.parameters())
        optim = torch.optim.SGD(
            param,
            lr=6e-2 * lr_factor,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class DCL(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.projection_head = heads.SimCLRProjectionHead(512, 512, 128)
        self.criterion = DCLLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2 * lr_factor, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


class DCLW(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)
        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )
        self.projection_head = heads.SimCLRProjectionHead(512, 512, 128)
        self.criterion = DCLWLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2 * lr_factor, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


from sklearn.cluster import KMeans


class SMoGModel(BenchmarkModule):
    def __init__(self, dataloader_kNN, num_classes):
        super().__init__(dataloader_kNN, num_classes)

        # create a ResNet backbone and remove the classification head
        resnet = ResNetGenerator("resnet-18")
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1], nn.AdaptiveAvgPool2d(1)
        )

        # create a model based on ResNet
        self.projection_head = heads.SMoGProjectionHead(512, 2048, 128)
        self.prediction_head = heads.SMoGPredictionHead(128, 2048, 128)
        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)
        utils.deactivate_requires_grad(self.backbone_momentum)
        utils.deactivate_requires_grad(self.projection_head_momentum)

        # smog
        self.n_groups = 300
        memory_bank_size = 10000
        self.memory_bank = memory_bank.MemoryBankModule(size=memory_bank_size)
        # create our loss
        group_features = torch.nn.functional.normalize(
            torch.rand(self.n_groups, 128), dim=1
        )
        self.smog = heads.SMoGPrototypes(group_features=group_features, beta=0.99)
        self.criterion = nn.CrossEntropyLoss()

    def _cluster_features(self, features: torch.Tensor) -> torch.Tensor:
        features = features.cpu().numpy()
        kmeans = KMeans(self.n_groups).fit(features)
        clustered = torch.from_numpy(kmeans.cluster_centers_).float()
        clustered = torch.nn.functional.normalize(clustered, dim=1)
        return clustered

    def _reset_group_features(self):
        # see https://arxiv.org/pdf/2207.06167.pdf Table 7b)
        features = self.memory_bank.bank
        group_features = self._cluster_features(features.t())
        self.smog.set_group_features(group_features)

    def _reset_momentum_weights(self):
        # see https://arxiv.org/pdf/2207.06167.pdf Table 7b)
        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)
        utils.deactivate_requires_grad(self.backbone_momentum)
        utils.deactivate_requires_grad(self.projection_head_momentum)

    def training_step(self, batch, batch_idx):
        if self.global_step > 0 and self.global_step % 300 == 0:
            # reset group features and weights every 300 iterations
            self._reset_group_features()
            self._reset_momentum_weights()
        else:
            # update momentum
            utils.update_momentum(self.backbone, self.backbone_momentum, 0.99)
            utils.update_momentum(
                self.projection_head, self.projection_head_momentum, 0.99
            )

        (x0, x1), _, _ = batch

        if batch_idx % 2:
            # swap batches every second iteration
            x0, x1 = x1, x0

        x0_features = self.backbone(x0).flatten(start_dim=1)
        x0_encoded = self.projection_head(x0_features)
        x0_predicted = self.prediction_head(x0_encoded)
        x1_features = self.backbone_momentum(x1).flatten(start_dim=1)
        x1_encoded = self.projection_head_momentum(x1_features)

        # update group features and get group assignments
        assignments = self.smog.assign_groups(x1_encoded)
        group_features = self.smog.get_updated_group_features(x0_encoded)
        logits = self.smog(x0_predicted, group_features, temperature=0.1)
        self.smog.set_group_features(group_features)

        loss = self.criterion(logits, assignments)

        # use memory bank to periodically reset the group features with k-means
        self.memory_bank(x0_encoded, update=True)

        return loss

    def configure_optimizers(self):
        params = (
            list(self.backbone.parameters())
            + list(self.projection_head.parameters())
            + list(self.prediction_head.parameters())
        )
        optim = torch.optim.SGD(
            params,
            lr=0.01,
            momentum=0.9,
            weight_decay=1e-6,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


models = [
    BarlowTwinsModel,
    BYOLModel,
    SimSiamModel,
    VICRegModel,
]
bench_results = dict()

experiment_version = None
# loop through configurations and train models
for BenchmarkModel in models:
    runs = []
    model_name = BenchmarkModel.__name__.replace("Model", "")
    for seed in range(n_runs):
        pl.seed_everything(seed)
        dataset_train_ssl = create_dataset_train_ssl(BenchmarkModel)
        dataloader_train_ssl, dataloader_train_kNN, dataloader_test = get_data_loaders(
            batch_size=batch_size, dataset_train_ssl=dataset_train_ssl
        )
        benchmark_model = BenchmarkModel(dataloader_train_kNN, classes)

        # Save logs to: {CWD}/benchmark_logs/cifar10/{experiment_version}/{model_name}/
        # If multiple runs are specified a subdirectory for each run is created.
        sub_dir = model_name if n_runs <= 1 else f"{model_name}/run{seed}"
        logger = TensorBoardLogger(
            save_dir=os.path.join(logs_root_dir, "cifar10"),
            name="",
            sub_dir=sub_dir,
            version=experiment_version,
        )
        if experiment_version is None:
            # Save results of all models under same version directory
            experiment_version = logger.version
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            dirpath=os.path.join(logger.log_dir, "checkpoints")
        )
        trainer = pl.Trainer(
            max_epochs=max_epochs,
            devices=devices,
            accelerator=accelerator,
            default_root_dir=logs_root_dir,
            strategy=strategy,
            sync_batchnorm=sync_batchnorm,
            logger=logger,
            callbacks=[checkpoint_callback],
        )
        start = time.time()
        trainer.fit(
            benchmark_model,
            train_dataloaders=dataloader_train_ssl,
            val_dataloaders=dataloader_test,
        )
        end = time.time()
        run = {
            "model": model_name,
            "batch_size": batch_size,
            "epochs": max_epochs,
            "max_accuracy": benchmark_model.max_accuracy,
            "runtime": end - start,
            "gpu_memory_usage": torch.cuda.max_memory_allocated(),
            "seed": seed,
        }
        runs.append(run)
        print(run)

        # delete model and trainer + free up cuda memory
        del benchmark_model
        del trainer
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    bench_results[model_name] = runs

# print results table
header = (
    f"| {'Model':<13} | {'Batch Size':>10} | {'Epochs':>6} "
    f"| {'KNN Test Accuracy':>18} | {'Time':>10} | {'Peak GPU Usage':>14} |"
)
print("-" * len(header))
print(header)
print("-" * len(header))
for model, results in bench_results.items():
    runtime = np.array([result["runtime"] for result in results])
    runtime = runtime.mean() / 60  # convert to min
    accuracy = np.array([result["max_accuracy"] for result in results])
    gpu_memory_usage = np.array([result["gpu_memory_usage"] for result in results])
    gpu_memory_usage = gpu_memory_usage.max() / (1024**3)  # convert to gbyte

    if len(accuracy) > 1:
        accuracy_msg = f"{accuracy.mean():>8.3f} +- {accuracy.std():>4.3f}"
    else:
        accuracy_msg = f"{accuracy.mean():>18.3f}"

    print(
        f"| {model:<13} | {batch_size:>10} | {max_epochs:>6} "
        f"| {accuracy_msg} | {runtime:>6.1f} Min "
        f"| {gpu_memory_usage:>8.1f} GByte |",
        flush=True,
    )
print("-" * len(header))


ImportError: ignored